In [1]:
from datasets import load_dataset
import pandas as pd
import os
os.environ['http_proxy'] = 'http://172.19.64.1:7890'
os.environ['https_proxy'] = 'http://172.19.64.1:7890'

# Load the dataset
dataset = load_dataset("neuralwork/arxiver")

# Convert to pandas DataFrame for easier viewing
df = dataset['train'].to_pandas()

# Show the first 5 papers with title and abstract
# print(df['markdown'].iloc[0])

# df[['markdown']].to_csv('raw_paper_data/markdown_only.csv', index=False)
type(df['markdown'].values)

numpy.ndarray

In [2]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
separators = ["\n\n", "\n", " ", "", "."]

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(separators=separators, chunk_size=1000, chunk_overlap=5)

# Split your docs into texts
texts_paper = text_splitter.create_documents(df['markdown'].values)

# Embeddings
hg_embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma_rag/'
economic_langchain_chroma = Chroma.from_documents(
    documents=texts_paper,
    collection_name="humanizer_data",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)